<a href="https://colab.research.google.com/github/Sarenela/cnn-occlusion-sensitivity-analisys/blob/main/notebook/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importy


In [ ]:
!git clone https://github.com/facebookresearch/segment-anything.git
!git clone https://github.com/IDEA-Research/groundingdino.git
!git clone https://github.com/IDEA-Research/Grounded-Segment-Anything.git

%cd /content/groundingdino
!pip install -r requirements.txt

%cd /content/segment-anything
!pip install -r requirements.txt

%cd /content/Grounded-Segment-Anything
!pip install -r requirements.txt


fatal: destination path 'segment-anything' already exists and is not an empty directory.
fatal: destination path 'groundingdino' already exists and is not an empty directory.
fatal: destination path 'Grounded-Segment-Anything' already exists and is not an empty directory.
/content/groundingdino
/content/segment-anything
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


KeyboardInterrupt: 

In [3]:
!pip install -q rembg
from rembg import remove

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 6.0 MB/s eta 0:00:00


In [ ]:
import sys

sys.path.extend([
    "/content/segment-anything",
    "/content/groundingdino",
    "/content/Grounded-Segment-Anything",
])

from pathlib import Path
import zipfile
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import resnet18, ResNet18_Weights, efficientnet_b0, EfficientNet_B0_Weights

from grounded_sam import build_sam, SamPredictor
from segment_anything import sam_model_registry, SamPredictor
from PIL import Image
import numpy as n

import matplotlib.pyplot as plt

from google.colab import files
from google.colab import drive



In [ ]:
!ls /content/Grounded-Segment-Anything

### Wczytywanie i przygotowanie danych

In [ ]:
drive.mount('/content/drive')


In [ ]:
data_dir = "/content/drive/My Drive/Colab Notebooks/data"
batch_size = 64

#Transformacje obrazu
#ResNet18_Weights.DEFAULT.transforms() = EfficientNet_B0_Weights.DEFAULT.transforms() = ImageClassification(
#     crop_size=[224]
#     resize_size=[256]
#     mean=[0.485, 0.456, 0.406]
#     std=[0.229, 0.224, 0.225]
#     interpolation=InterpolationMode.BILINEAR
# )
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

dataset = datasets.ImageFolder(root=data_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)


#Mapowanie klas ImageNet do cat/dog
#https://deeplearning.cms.waikato.ac.nz/user-guide/class-maps/IMAGENET/
cat_indices = list(range(281, 294))
dog_indices = list(range(151, 269))


### Przygotowanie danyc

### implementacje Occlusion i inne pomocnicze

In [ ]:
def apply_occlusion_square(image: torch.Tensor, occlusion_percent: float = 20, top: int = None,
                    left: int = None) -> torch.Tensor:
    c, h, w = image.shape
    occ_h = int(h * occlusion_percent / 100)
    occ_w = int(w * occlusion_percent / 100)

    if top is None:
        top = (h - occ_h) // 2
    if left is None:
        left = (w - occ_w) // 2

    occluded = image.clone()
    occluded[:, top:top + occ_h, left:left + occ_w] = 0  #black square

    return occluded



def apply_background_removal(image: torch.Tensor, sam_model, sam_processor, device='cuda') -> torch.Tensor:
    # Denormalizacja obrazu do PIL
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    img_denorm = image * std + mean
    img_denorm = torch.clamp(img_denorm, 0, 1)

    # Konwersja do PIL Image
    img_pil = Image.fromarray((img_denorm.permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8))

    try:
        # Generowanie wielu punktów w różnych miejscach obrazu
        h, w = img_pil.size
        # Siatka punktów: 3x3 grid po całym obrazie
        points = []
        for i in range(3):
            for j in range(3):
                x = int(w * (i + 1) / 4)
                y = int(h * (j + 1) / 4)
                points.append([x, y])

        input_points = [points]

        # Przygotowanie inputów dla SAM
        inputs = sam_processor(img_pil, input_points=input_points, return_tensors="pt").to(device)

        with torch.no_grad():
            outputs = sam_model(**inputs)

        # Ekstrakcja masek
        masks = sam_processor.image_processor.post_process_masks(
            outputs.pred_masks.cpu(),
            inputs["original_sizes"].cpu(),
            inputs["reshaped_input_sizes"].cpu()
        )

        # Wybór maski z największą powierzchnią (zazwyczaj główny obiekt)
        all_masks = masks[0][0]  # Wszystkie maski dla wszystkich punktów
        mask_areas = [mask.sum().item() for mask in all_masks]

        # Filtruj maski - nie chcemy ani bardzo małych (szum) ani bardzo dużych (całe tło)
        valid_masks = []
        total_pixels = all_masks[0].numel()
        for i, (mask, area) in enumerate(zip(all_masks, mask_areas)):
            area_ratio = area / total_pixels
            # Obiekt powinien zajmować 10-80% obrazu
            if 0.1 < area_ratio < 0.8:
                valid_masks.append((i, area, mask))

        if valid_masks:
            # Wybierz maskę o największej powierzchni spośród validnych
            best_idx, best_area, best_mask = max(valid_masks, key=lambda x: x[1])
            mask = best_mask.numpy()
        else:
            # Fallback: użyj maski o średniej wielkości
            mask = all_masks[len(all_masks)//2].numpy()

        # Resize maski do rozmiaru obrazu
        from scipy.ndimage import zoom
        mask_resized = zoom(mask, (224/mask.shape[0], 224/mask.shape[1]), order=1)
        mask_tensor = torch.from_numpy(mask_resized).float().to(image.device)

        # Zostaw tylko obiekt, reszta czarna
        mask_3d = mask_tensor.unsqueeze(0).expand_as(image)
        result = image * mask_3d

        return result

    except Exception as e:
        print(f"Błąd segmentacji: {e}, zwracam oryginalny obraz")
        return image


def predict_cat_dog(outputs):
    probs = torch.nn.functional.softmax(outputs, dim=1)
    top_idx = torch.argmax(probs, dim=1)
    predictions = []
    for idx in top_idx:
        if idx.item() in cat_indices:
            predictions.append("cat")
        elif idx.item() in dog_indices:
            predictions.append("dog")
        else:
            predictions.append("other")
    return predictions


def compute_accuracy(model, dataloader, occlusion_type="square", occlusion_percent=20,
                     sam_model=None, sam_processor=None):
    correct = 0
    total = 0

    for images, labels in dataloader:
        if occlusion_type == "square":
            images = torch.stack([apply_occlusion_square(img, occlusion_percent=occlusion_percent) for img in images])
        elif occlusion_type == "background_removal":
            if sam_model is None or sam_processor is None:
                raise ValueError("SAM model i processor są wymagane dla background_removal")
            images = torch.stack([apply_background_removal(img, sam_model, sam_processor, device) for img in images])

        images = images.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            outputs = model(images)

        preds = predict_cat_dog(outputs)
        label_names = ["cat" if l == 0 else "dog" for l in labels]

        for p, l in zip(preds, label_names):
            if p == l:
                correct += 1
            total += 1

    return correct / total


Wizualizacja przykładowych obrazów

In [ ]:
def denormalize_image(image):
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    img_denorm = image * std + mean
    img_denorm = torch.clamp(img_denorm, 0, 1)
    return img_denorm.permute(1, 2, 0).cpu().numpy()

# Zbierz wszystkie obrazy do jednej listy
all_images = []
all_labels = []
for images, labels in dataloader:
    all_images.append(images)
    all_labels.append(labels)
all_images = torch.cat(all_images, dim=0)
all_labels = torch.cat(all_labels, dim=0)

# Losowy wybór 20 obrazów
import random
random.seed(42)
indices = random.sample(range(len(all_images)), 20)

# Przygotuj obrazy z różnymi okluzjami
original_images = [all_images[i] for i in indices]
occluded_20_images = [apply_occlusion_square(all_images[i], occlusion_percent=20) for i in indices]
bg_removed_images = [apply_background_removal(all_images[i], sam_model, sam_processor, device) for i in indices]
labels_selected = [all_labels[i].item() for i in indices]

# Wykres 1: Okluzja kwadratowa 20%
fig, axes = plt.subplots(4, 5, figsize=(15, 12))
fig.suptitle('Okluzja kwadratowa 20%', fontsize=16)

for idx, (ax, img, label) in enumerate(zip(axes.flat, occluded_20_images, labels_selected)):
    img_display = denormalize_image(img)
    ax.imshow(img_display)
    ax.axis('off')
    label_name = "Cat" if label == 0 else "Dog"
    ax.set_title(f'{idx+1}. {label_name}', fontsize=10)

plt.tight_layout()
plt.show()

# Wykres 2: Usunięcie tła (SAM)
fig, axes = plt.subplots(4, 5, figsize=(15, 12))
fig.suptitle('Usunięcie tła (czarne) - SAM', fontsize=16)

for idx, (ax, img, label) in enumerate(zip(axes.flat, bg_removed_images, labels_selected)):
    img_display = denormalize_image(img)
    ax.imshow(img_display)
    ax.axis('off')
    label_name = "Cat" if label == 0 else "Dog"
    ax.set_title(f'{idx+1}. {label_name}', fontsize=10)

plt.tight_layout()
plt.show()

# Wykres 3: Porównanie obok siebie (5 przykładów)
fig, axes = plt.subplots(5, 3, figsize=(12, 20))
fig.suptitle('Porównanie metod okluzji (5 przykładów)', fontsize=16)

for row in range(5):
    # Oryginalny
    img_display = denormalize_image(original_images[row])
    axes[row, 0].imshow(img_display)
    axes[row, 0].axis('off')
    if row == 0:
        axes[row, 0].set_title('Oryginalny', fontsize=12, fontweight='bold')
    label_name = "Cat" if labels_selected[row] == 0 else "Dog"
    axes[row, 0].set_ylabel(f'{label_name}', fontsize=11, fontweight='bold')

    # Okluzja 20%
    img_display = denormalize_image(occluded_20_images[row])
    axes[row, 1].imshow(img_display)
    axes[row, 1].axis('off')
    if row == 0:
        axes[row, 1].set_title('Okluzja 20%', fontsize=12, fontweight='bold')

    # Usunięcie tła
    img_display = denormalize_image(bg_removed_images[row])
    axes[row, 2].imshow(img_display)
    axes[row, 2].axis('off')
    if row == 0:
        axes[row, 2].set_title('Usunięcie tła (SAM)', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()



### Ewaluacja

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Wczytanie modeli
model_resnet = resnet18(weights=ResNet18_Weights.DEFAULT).to(device)
model_resnet.eval()

model_efficientnet = efficientnet_b0(weights=EfficientNet_B0_Weights.DEFAULT).to(device)
model_efficientnet.eval()


print("Ładowanie modelu SAM...")
sam_model = SamModel.from_pretrained("facebook/sam-vit-base").to(device)
sam_processor = SamProcessor.from_pretrained("facebook/sam-vit-base")
sam_model.eval()
print("Model SAM załadowany!")

# Test 1: Okluzja kwadratowa
print("\n=== TEST 1: Okluzja kwadratowa ===")
occlusion_levels = list(range(0, 100, 5))
resnet_acc_square = []
effnet_acc_square = []

for occ in occlusion_levels:
    acc_r = compute_accuracy(model_resnet, dataloader, occlusion_type="square", occlusion_percent=occ)
    acc_e = compute_accuracy(model_efficientnet, dataloader, occlusion_type="square", occlusion_percent=occ)
    resnet_acc_square.append(acc_r)
    effnet_acc_square.append(acc_e)
    print(f"Okluzja {occ}% -> ResNet: {acc_r:.4f}, EfficientNet: {acc_e:.4f}")

# Test 2: Usunięcie tła (czarne tło)
print("\n=== TEST 2: Usunięcie tła (SAM) ===")
acc_r = compute_accuracy(model_resnet, dataloader, occlusion_type="background_removal",
                        sam_model=sam_model, sam_processor=sam_processor)
acc_e = compute_accuracy(model_efficientnet, dataloader, occlusion_type="background_removal",
                        sam_model=sam_model, sam_processor=sam_processor)
resnet_acc_bg = acc_r
effnet_acc_bg = acc_e
print(f"Czarne tło -> ResNet: {acc_r:.4f}, EfficientNet: {acc_e:.4f}")

# Wykresy
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Wykres 1: Okluzja kwadratowa
axes[0].plot(occlusion_levels, resnet_acc_square, marker='o', label='ResNet18')
axes[0].plot(occlusion_levels, effnet_acc_square, marker='o', label='EfficientNet-B0')
axes[0].set_title("Accuracy - Okluzja kwadratowa")
axes[0].set_xlabel("Procent okluzji")
axes[0].set_ylabel("Accuracy")
axes[0].grid(True)
axes[0].legend()

# Wykres 2: Różnica dla okluzji kwadratowej
diff_square = [e - r for e, r in zip(effnet_acc_square, resnet_acc_square)]
axes[1].plot(occlusion_levels, diff_square, marker='o', color='green')
axes[1].set_title("Różnica accuracy (EfficientNet - ResNet)")
axes[1].set_xlabel("Procent okluzji")
axes[1].set_ylabel("Różnica accuracy")
axes[1].axhline(0, color='gray', linestyle='--')
axes[1].grid(True)

plt.tight_layout()
plt.show()

# Wykres porównania z usunięciem tła
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
methods = ['Baseline\n(bez okluzji)', 'Okluzja 50%\n(kwadrat)', 'Usunięcie tła\n(SAM)']
resnet_scores = [resnet_acc_square[0], resnet_acc_square[10], resnet_acc_bg]
effnet_scores = [effnet_acc_square[0], effnet_acc_square[10], effnet_acc_bg]

x = np.arange(len(methods))
width = 0.35

bars1 = ax.bar(x - width/2, resnet_scores, width, label='ResNet18', alpha=0.8)
bars2 = ax.bar(x + width/2, effnet_scores, width, label='EfficientNet-B0', alpha=0.8)

ax.set_ylabel('Accuracy')
ax.set_title('Porównanie metod okluzji')
ax.set_xticks(x)
ax.set_xticklabels(methods)
ax.legend()
ax.grid(True, alpha=0.3)

# Dodaj wartości na słupkach
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}',
                ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

# Porównanie metod
print("\n=== PODSUMOWANIE ===")
print(f"Baseline (bez okluzji):")
print(f"  ResNet: {resnet_acc_square[0]:.4f}, EfficientNet: {effnet_acc_square[0]:.4f}")
print(f"\nOkluzja kwadratowa 50%:")
print(f"  ResNet: {resnet_acc_square[10]:.4f}, EfficientNet: {effnet_acc_square[10]:.4f}")
print(f"\nUsunięcie tła (czarne):")
print(f"  ResNet: {resnet_acc_bg:.4f}, EfficientNet: {effnet_acc_bg:.4f}")